<a href="https://colab.research.google.com/github/zosiak00/uczenie_maszynowe_ZK/blob/case3/zad3_ZK_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

W poprzednich wersjach pliku w statystykach pojawiąły się wartości null, dlatego zrobiłam notatnik jeszcze raz, z trochę innym podejściem

Dodatkowo w Githubie widzę, że nie wygenerował się wykres, który w Colabie się generujem więc wrzucam link: https://colab.research.google.com/drive/1V_MRbi3zDSmNq9OoYsw1d1IuLRgefyyO?usp=sharing

In [2]:
!pip3 install ta
!pip3 install backtesting

  Using cached Backtesting-0.3.3-py3-none-any.whl


In [43]:
pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 4.0 MB/s eta 0:00:00


In [28]:
import yfinance as yf
import pandas as pd
import ta
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from backtesting.lib import crossover


In [29]:
ticker = 'AAPL'
data = yf.download(ticker, start='2014-01-01', end='2024-05-06')

[*********************100%%**********************]  1 of 1 completed


In [30]:
data['SMA20'] = ta.trend.sma_indicator(data['Close'], window=20)
data['SMA50'] = ta.trend.sma_indicator(data['Close'], window=50)
data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
data['MACD'] = ta.trend.macd_diff(data['Close'])
data['StochK'] = ta.momentum.stoch(data['High'], data['Low'], data['Close'], window=14, smooth_window=3)
data['StochD'] = ta.momentum.stoch_signal(data['High'], data['Low'], data['Close'], window=14, smooth_window=3)
data['CCI'] = ta.trend.cci(data['High'], data['Low'], data['Close'], window=20)
data['ATR'] = ta.volatility.average_true_range(data['High'], data['Low'], data['Close'], window=14)
data['MFI'] = ta.volume.money_flow_index(data['High'], data['Low'], data['Close'], data['Volume'], window=14)
data = data.dropna()
data['Signal'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)

<ipython-input-30-0890dc6aa6a0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)


In [31]:
# podział danych na tremnigowy i testowy
train_data = data['2014-01-01':'2023-12-31']
test_data = data['2024-01-01':'2024-05-06']

X_train = train_data[['SMA20', 'SMA50', 'RSI', 'MACD', 'StochK', 'StochD', 'CCI', 'ATR', 'MFI']]
y_train = train_data['Signal']
X_test = test_data[['SMA20', 'SMA50', 'RSI', 'MACD', 'StochK', 'StochD', 'CCI', 'ATR', 'MFI']]
y_test = test_data['Signal']

In [32]:
# Hiperparametryzacja modelu
param_grid = {
     'n_estimators': [100, 200, 300],
     'learning_rate': [0.1, 0.01, 0.05],
     'colsample_bytree': [0.5, 0.75, 1.0],
     'max_depth': range(1,5,9)
}

In [33]:
model = XGBClassifier()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.5, 0.75, 1.0],
                         'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(1, 5, 9),
                         'n_estimators': [100, 200, 300]})

In [34]:
best_model = grid_search.best_estimator_

model = XGBClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [40]:
from sklearn.metrics import balanced_accuracy_score
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Ocena modelu
train_accuracy = balanced_accuracy_score(y_train, y_pred_train)
test_accuracy = balanced_accuracy_score(y_test, y_pred_test)

print(train_accuracy)
print(test_accuracy)

0.509509305591266
0.5


In [35]:
class MLStrategy(Strategy):
    def init(self):
        self.sma20 = self.I(ta.trend.sma_indicator, pd.Series(self.data.Close), window=20)
        self.sma50 = self.I(ta.trend.sma_indicator, pd.Series(self.data.Close), window=50)
        self.rsi = self.I(ta.momentum.rsi, pd.Series(self.data.Close), window=14)
        self.macd = self.I(ta.trend.macd_diff, pd.Series(self.data.Close))
        self.stochk = self.I(ta.momentum.stoch, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.stochd = self.I(ta.momentum.stoch_signal, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.cci = self.I(ta.trend.cci, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=20)
        self.atr = self.I(ta.volatility.average_true_range, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.mfi = self.I(ta.volume.money_flow_index, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), pd.Series(self.data.Volume), window=14)

    def next(self):
        features = [
            self.sma20[-1], self.sma50[-1],
            self.rsi[-1], self.macd[-1],
            self.stochk[-1], self.stochd[-1],
            self.cci[-1], self.atr[-1],
            self.mfi[-1]
        ]

        signal = model.predict([features])[0]

        if signal == 1 and not self.position:
            self.buy()
        elif signal == 0 and self.position:
            self.sell()

In [45]:
# Bactesting
bt = Backtest(test_data, MLStrategy, cash=100000, commission=0.002)
stats = bt.run()
bt.plot()
plt.show()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [37]:
print(stats)

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                   40.697674
Equity Final [$]                108825.253291
Equity Peak [$]                 108825.253291
Return [%]                           8.825253
Buy & Hold Return [%]               -1.217407
Return (Ann.) [%]                    28.12283
Volatility (Ann.) [%]               26.640482
Sharpe Ratio                         1.055643
Sortino Ratio                        2.691485
Calmar Ratio                         3.676011
Max. Drawdown [%]                   -7.650366
Avg. Drawdown [%]                   -7.650366
Max. Drawdown Duration       44 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       8.825986
Worst Trade [%]                      8.825986
Avg. Trade [%]                    

In [38]:
trades = stats['_trades']
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,583,51,85,171.512338,186.649994,8825.253291,0.08826,2024-03-15,2024-05-03,49 days


Polecana strategia inwestycyjna: zakup 2024-03-15, sprzedaż 2024-05-03

Zysk: 8.826 %